# Generating random GORGIAS code

The objective of this notebook is to generate an random but syntaxically correct GORGIAS code, we won't give importance to the semantics behind each code.

In [ ]:
import random

actions = [
    # Work-related actions
    "attend_meeting", "finish_report", "reply_emails", "give_presentation",
    
    # Social & leisure activities
    "go_to(restaurant)", "go_to(cinema)", "go_to(park)", "go_to(theater)",
    "visit_family", "attend_concert", "travel_abroad", "go_shopping",

    # Health & exercise
    "go_gym", "morning_run", "yoga_session", "visit_doctor",

    # Daily tasks
    "buy_groceries", "clean_house", "cook_dinner", "read_book",

    # Transportation
    "take_bus", "ride_bike", "drive_car", "book_flight"
]

facts = [
    # Work-related
    "urgent_deadline", "important_meeting", "boss_in_office", "team_project_due",

    # Personal situations
    "feeling_sick", "birthday_today", "wedding_anniversary", "friend_in_town",

    # Weather conditions
    "good_weather", "rainy_day", "snowstorm", "extreme_heat",

    # Time-based events
    "weekend", "holiday_season", "morning_rush", "night_time",

    # Social dynamics
    "invitation_from_friend", "family_gathering", "new_restaurant_to_try", "concert_nearby",

    # Financial considerations
    "low_budget", "got_bonus", "discount_on_flight", "expensive_event"
]


We will classify any GORGIAS as beginner level as the following : 
- simple arguments with clear rules and priorities
- no recursion with minimal dependencies
- a maximum of 1 or 2 layers of preferences
- no `complement()/2`, abducible or defeasible components

In [ ]:
def generate_beginner_gorgias():
    action1, action2 = random.sample(actions, 2)
    shared_condition = random.choice(facts)
    
    remaining_facts = [fact for fact in facts if fact != shared_condition]
    p2_condition = random.choice(remaining_facts)

    remaining_facts = [fact for fact in remaining_facts if fact != p2_condition]
    c2_condition = random.choice(remaining_facts)

    rule1 = f"rule(r1, {action1}, []) :- {shared_condition}."
    rule2 = f"rule(r2, {action2}, []) :- {shared_condition}."

    pref1 = f"rule(p1, prefer(r1, r2), [])."

    depth = random.choice([1, 2, 3])  # 1 = just p1, 2 = add p2 & c1, 3 = full depth

    rules = [rule1, rule2, pref1]

    if depth >= 2:
        pref2 = f"rule(p2, prefer(r2, r1), []) :- {p2_condition}."
        conflict1 = "rule(c1, prefer(p2, p1), [])."
        rules.extend([pref2, conflict1])

    if depth == 3:
        conflict2 = f"rule(c2, prefer(p1, p2), []) :- {c2_condition}."
        conflict3 = "rule(c3, prefer(c2, c1), [])."
        rules.extend([conflict2, conflict3])

    return "\n".join(rules)

For the intermediate level we will add the `complement()/2` having multi-level 

In [ ]:
num_examples = 3
gorgias_examples = [generate_beginner_gorgias() for _ in range(num_examples)]

for i, example in enumerate(gorgias_examples, 1):
    print(f"### Example {i} ###\n{example}\n")